## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 

url = 'https://raw.githubusercontent.com/sophiastewart202/Deep_Learning_Challenge/main/Resources/charity_data.csv'
application_df = pd.read_csv(url)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=['EIN', 'NAME'])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_value_counts = df['APPLICATION_TYPE'].value_counts()

app_counts = pd.DataFrame(app_value_counts)
app_counts

,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156
T13,66


In [5]:
# Determine cutoff value
total = app_counts['APPLICATION_TYPE'].sum()
highest_counts = app_counts.loc[app_counts['APPLICATION_TYPE']>500].sum()

highest_counts/total

APPLICATION_TYPE    0.991953
dtype: float64

In [6]:
# Create a list of application types to be replaced

application_types_to_replace = app_counts.loc[app_counts['APPLICATION_TYPE']<500].index.to_list()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_counts = df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4500        1
C1236        1
C2380        1
C2170        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts >1
class_counts = pd.DataFrame(class_counts)
class_counts.loc[class_counts['CLASSIFICATION']>1]

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114


In [9]:
# Determine cutoff value
total = class_counts['CLASSIFICATION'].sum()
highest_counts = class_counts.loc[class_counts['CLASSIFICATION']>1000].sum()

highest_counts/total

CLASSIFICATION    0.93408
dtype: float64

In [10]:
# Create a list of classifications to be replaced
classifications_to_replace = class_counts.loc[class_counts['CLASSIFICATION']<1000].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
num_df = pd.get_dummies(df)
num_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
target = num_df['IS_SUCCESSFUL']
features = num_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=26)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
X_train_scaled.shape

(25724, 43)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=int(X_train_scaled.shape[1])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 220       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6303 - accuracy: 0.6769
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5770 - accuracy: 0.7181
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5658 - accuracy: 0.7247
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5614 - accuracy: 0.7264
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5592 - accuracy: 0.7265
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5583 - accuracy: 0.7269
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5577 - accuracy: 0.7275
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5565 - accuracy: 0.7276
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7279
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5544 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7282
Loss: 0.5555833578109741, Accuracy: 0.7281632423400879


**Model 2**

---



In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=int(X_train_scaled.shape[1])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                440       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6155 - accuracy: 0.6953
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5696 - accuracy: 0.7279
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5614 - accuracy: 0.7285
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7292
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5557 - accuracy: 0.7295
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5547 - accuracy: 0.7307
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5538 - accuracy: 0.7305
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5531 - accuracy: 0.7300
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7301
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5518 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5555 - accuracy: 0.7264
Loss: 0.5554910898208618, Accuracy: 0.7264139652252197


**Model 3**

---



In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=int(X_train_scaled.shape[1])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 15)                660       
_________________________________________________________________
dense_7 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 6         
Total params: 881
Trainable params: 881
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6134 - accuracy: 0.7016
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5642 - accuracy: 0.7271
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5576 - accuracy: 0.7276
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5548 - accuracy: 0.7305
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7306
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7306
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7300
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5498 - accuracy: 0.7310
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7318
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5587 - accuracy: 0.7272
Loss: 0.5586581230163574, Accuracy: 0.7272303104400635


**Model 4**

---



In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu", input_dim=int(X_train_scaled.shape[1])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 25)                1100      
_________________________________________________________________
dense_11 (Dense)             (None, 15)                390       
_________________________________________________________________
dense_12 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
Total params: 1,661
Trainable params: 1,661
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5831 - accuracy: 0.7100
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7254
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7293
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7295
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7310
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7301
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7322
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7327
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7332
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5451 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5586 - accuracy: 0.7284
Loss: 0.5586429238319397, Accuracy: 0.728396475315094


**Model 5**

---



In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=int(X_train_scaled.shape[1])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 15)                660       
_________________________________________________________________
dense_15 (Dense)             (None, 25)                400       
_________________________________________________________________
dense_16 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_17 (Dense)             (None, 15)                390       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16        
Total params: 2,116
Trainable params: 2,116
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

In [33]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5853 - accuracy: 0.7096
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5582 - accuracy: 0.7263
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7300
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5530 - accuracy: 0.7293
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5509 - accuracy: 0.7308
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7312
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7322
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7327
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7318
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accura

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5637 - accuracy: 0.7285
Loss: 0.5636670589447021, Accuracy: 0.7285131216049194


In [36]:
# Export our model to HDF5 file
nn.save('charity_funding_predictor.h5')